# Engenharia de Atributos

In [54]:
import pandas as pd

In [55]:
merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')

C:\Users\Matheus\AppData\Local\Temp\ipykernel_12880\3999875462.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_dataset = pd.read_csv('../data/processed/merged_dataset.csv')


Convertendo as variáveis para os seus respectivos tipos corretos 

In [56]:
# Conversão dos tipos numéricos e categóricos
merged_dataset = merged_dataset.astype({
    'ID_CLIENTE': 'int64',
    'VALOR_A_PAGAR': 'float64',
    'TAXA': 'float64',
    'RENDA_MES_ANTERIOR': 'float64',
    'NO_FUNCIONARIOS': 'int64',
    'FLAG_PF': 'int', 
    'DIAS_ATRASO': 'Int64',
    'TARGET_INADIMPLENCIA': 'int',
    'SEGMENTO_INDUSTRIAL': 'category',
    'PORTE': 'category',
    'CEP_2_DIG': 'category',
    'DIAS_ADIANTAMENTO': 'int'
})

# Conversão de datas
merged_dataset['SAFRA_REF'] = pd.to_datetime(
    merged_dataset['SAFRA_REF'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(
    merged_dataset['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_PAGAMENTO'] = pd.to_datetime(
    merged_dataset['DATA_PAGAMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_VENCIMENTO'] = pd.to_datetime(
    merged_dataset['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce'
)

merged_dataset['DATA_CADASTRO'] = pd.to_datetime(
    merged_dataset['DATA_CADASTRO'], format='%Y-%m-%d', errors='coerce'
)


In [57]:
merged_dataset.dtypes

ID_CLIENTE                         int64
SAFRA_REF                 datetime64[ns]
DATA_EMISSAO_DOCUMENTO    datetime64[ns]
DATA_PAGAMENTO            datetime64[ns]
DATA_VENCIMENTO           datetime64[ns]
VALOR_A_PAGAR                    float64
TAXA                             float64
RENDA_MES_ANTERIOR               float64
NO_FUNCIONARIOS                    int64
DATA_CADASTRO             datetime64[ns]
FLAG_PF                            int64
SEGMENTO_INDUSTRIAL             category
PORTE                           category
CEP_2_DIG                       category
DIAS_ATRASO                        Int64
TARGET_INADIMPLENCIA               int64
DIAS_ADIANTAMENTO                  int64
dtype: object

Criando novas features (pense nelas)

Tempo de cadastro até vencimento: mede o tempo entre o início da relação com o cliente e a fatura. Clientes mais novos (meno consolidado.

In [58]:
merged_dataset['TEMPO_CADASTRO_PARA_VENCIMENTO'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_CADASTRO']).dt.days

Prazo para o pagamento: ajuda a entender o tempo que o cliente teve para se preparar para o pagamento. Prazos curtos podem influenciar na inadimplência.



In [59]:
merged_dataset['PRAZO_PAGAMENTO'] = (merged_dataset['DATA_VENCIMENTO'] - merged_dataset['DATA_EMISSAO_DOCUMENTO']).dt.days

Mês/Ano da SAFRA_REF: capta padrões sazonais (ex: pode haver mais inadimplência no final do ano ou em meses com impostos).

In [60]:
merged_dataset['MES_SAFRA'] = merged_dataset['SAFRA_REF'].dt.month
merged_dataset['ANO_SAFRA'] = merged_dataset['SAFRA_REF'].dt.year


In [61]:
merged_dataset

,ID_CLIENTE,SAFRA_REF,DATA_EMISSAO_DOCUMENTO,DATA_PAGAMENTO,DATA_VENCIMENTO,VALOR_A_PAGAR,TAXA,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS,DATA_CADASTRO,...,SEGMENTO_INDUSTRIAL,PORTE,CEP_2_DIG,DIAS_ATRASO,TARGET_INADIMPLENCIA,DIAS_ADIANTAMENTO,TEMPO_CADASTRO_PARA_VENCIMENTO,PRAZO_PAGAMENTO,MES_SAFRA,ANO_SAFRA
0,8784237149961904,2018-09-01,2018-09-04,2018-09-24,2018-09-24,59610.76,5.99,300502.0,107,2011-02-14,...,Comércio,PEQUENO,27,0,0,0,2779,20,9,2018
1,8784237149961904,2018-09-01,2018-09-06,2018-09-24,2018-09-24,39398.06,5.99,300502.0,107,2011-02-14,...,Comércio,PEQUENO,27,0,0,0,2779,18,9,2018
2,8784237149961904,2018-09-01,2018-09-09,2018-09-25,2018-09-25,55416.75,5.99,300502.0,107,2011-02-14,...,Comércio,PEQUENO,27,0,0,0,2780,16,9,2018
3,8784237149961904,2018-09-01,2018-09-11,2018-09-27,2018-09-27,11751.35,5.99,300502.0,107,2011-02-14,...,Comércio,PEQUENO,27,0,0,0,2782,16,9,2018
4,8784237149961904,2018-09-01,2018-09-17,2018-10-03,2018-10-03,35985.00,5.99,300502.0,107,2011-02-14,...,Comércio,PEQUENO,27,0,0,0,2788,16,9,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72485,9206030810342980458,2021-05-01,2021-05-16,2021-06-01,2021-06-01,107321.36,11.99,256133.0,109,2005-08-08,...,Serviços,MEDIO,91,0,0,0,5776,16,5,2021
72486,9206030810342980458,2021-05-01,2021-05-23,2021-06-08,2021-06-08,38372.91,5.99,256133.0,109,2005-08-08,...,Serviços,MEDIO,91,0,0,0,5783,16,5,2021
72487,9206030810342980458,2021-06-01,2021-06-16,2021-07-02,2021-07-02,43030.50,5.99,463963.0,105,2005-08-08,...,Serviços,MEDIO,91,0,0,0,5807,16,6,2021
72488,9206030810342980458,2021-06-01,2021-06-18,2021-07-05,2021-07-05,107318.81,5.99,463963.0,105,2005-08-08,...,Serviços,MEDIO,91,0,0,0,5810,17,6,2021
